**CSI 5138 Homework 3**

Use Vanilla RNN and LSTM to for text classification and sentiment analysis on a standard dataset of movie reviews.

# Setup

In [111]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
import gensim

import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Embedding
from keras.layers import Input
from keras.layers import Dense, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
#from keras import backend as K

import os
import numpy as np

from pprint import pprint

# IMDB Movie Review Dataset, Test and Training Data

In [24]:
def processReviews(paths):
    texts = []
    ratings = []
    
    for path in paths:
        for file in os.listdir(path):
            # get review
            rating = file.split('_')[1]
            rating = rating.split('.')[0]
            file = os.path.join(path, file)
            with open(file, "r", encoding='utf-8') as f:
                text = []
                for line in f:
                    # do some pre-processing and combine list of words for each review text             
                    text += gensim.utils.simple_preprocess(line)
                texts.append(text)
                ratings.append(rating)
        
    return [texts, ratings]

In [ ]:
Xtrain, ytrain = processReviews(["./aclImdb/train/neg/", "./aclImdb/train/pos/"])
Xtest, ytest = processReviews(["./aclImdb/test/neg/", "./aclImdb/test/pos/"])

In [31]:
print(Xtrain[0])
print(ytrain[0])
print(Xtest[0])
print(ytest[0])
print(Xtrain[-1])
print(ytrain[-1])
print(Xtest[-1])
print(ytest[-1])
print("# Xtrain: ", len(Xtrain))
print("# ytrain: ", len(ytrain))
print("# Xtest: ", len(Xtest))
print("# ytest: ", len(ytest))

['story', 'of', 'man', 'who', 'has', 'unnatural', 'feelings', 'for', 'pig', 'starts', 'out', 'with', 'opening', 'scene', 'that', 'is', 'terrific', 'example', 'of', 'absurd', 'comedy', 'formal', 'orchestra', 'audience', 'is', 'turned', 'into', 'an', 'insane', 'violent', 'mob', 'by', 'the', 'crazy', 'chantings', 'of', 'it', 'singers', 'unfortunately', 'it', 'stays', 'absurd', 'the', 'whole', 'time', 'with', 'no', 'general', 'narrative', 'eventually', 'making', 'it', 'just', 'too', 'off', 'putting', 'even', 'those', 'from', 'the', 'era', 'should', 'be', 'turned', 'off', 'the', 'cryptic', 'dialogue', 'would', 'make', 'shakespeare', 'seem', 'easy', 'to', 'third', 'grader', 'on', 'technical', 'level', 'it', 'better', 'than', 'you', 'might', 'think', 'with', 'some', 'good', 'cinematography', 'by', 'future', 'great', 'vilmos', 'zsigmond', 'future', 'stars', 'sally', 'kirkland', 'and', 'frederic', 'forrest', 'can', 'be', 'seen', 'briefly']
3
['once', 'again', 'mr', 'costner', 'has', 'dragged', 

In [64]:
X = list(Xtrain + Xtest)
y = list(ytrain + ytest)
print("# X: ", len(X))
print("# y: ", len(y))

# X:  50000
# y:  50000


# Word embedding vectors

### Try Keras with Glove

In [98]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences)

labels = to_categorical(np.asarray(y))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)
print(labels[0])
# review on a 1-10 scale
# indices 0-3 are neg reviews <= 4, indices 6-9 are pos reviews >=7
labels = labels[:,1:]
print(labels[0])

Found 99476 unique tokens.
Shape of data tensor: (50000, 2380)
Shape of label tensor: (50000, 11)
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


In [113]:
x_train, y_train = data[:25000], labels[:25000]
x_val, y_val = data[25000:], labels[25000:]

In [114]:
print("# x_train: ", len(x_train))
print("# x_test: ", len(x_val))

# x_train:  25000
# x_test:  25000


In [ ]:
# processing train and test separetly
# train
# Found 73293 unique tokens.
# Shape of data tensor: (25000, 2380)
# Shape of label tensor: (25000, 11)
# [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]

# test
# Found 72280 unique tokens.
# Shape of data tensor: (25000, 2265)
# Shape of label tensor: (25000, 11)
# [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]

In [58]:
embeddings_index = {}
glove_file = './glove.6B/glove.6B.100d.txt'

with open(glove_file, "r", encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [69]:
EMBEDDING_DIM=100
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [118]:
print(len(word_index))

99476


In [101]:
#print(list(word_index.items())[100])
print(word_index['king'])
print(word_index['queen'])
print(embedding_matrix[682])
print(embedding_matrix[1577])
print(embedding_matrix[101])
print(embedding_matrix[0])

682
1577
[-0.32306999 -0.87616003  0.21977     0.25268     0.22976001  0.73879999
 -0.37954    -0.35306999 -0.84368998 -1.11129999 -0.30265999  0.33177999
 -0.25113001  0.30447999 -0.077491   -0.89815003  0.092496   -1.14069998
 -0.58323997  0.66869003 -0.23122001 -0.95854998  0.28262001 -0.078848
  0.75314999  0.26583999  0.34220001 -0.33949     0.95608002  0.065641
  0.45747     0.39835     0.57964998  0.39267001 -0.21851     0.58794999
 -0.55998999  0.63367999 -0.043983   -0.68730998 -0.37841001  0.38025999
  0.61641002 -0.88269001 -0.12346    -0.37928    -0.38317999  0.23868001
  0.66850001 -0.43320999 -0.11065     0.081723    1.15690005  0.78957999
 -0.21223    -2.3211     -0.67806     0.44560999  0.65706998  0.1045
  0.46217     0.19912     0.25802001  0.057194    0.53443003 -0.43133
 -0.34311     0.59789002 -0.58416998  0.068995    0.23943999 -0.85180998
  0.30379    -0.34176999 -0.25746    -0.031101   -0.16285001  0.45168999
 -0.91627002  0.64521003  0.73281002 -0.22752     0.3

In [103]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=False)

In [ ]:
MAX_SEQUENCE_LENGTH = 100
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)  # global max pooling
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(10, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

# happy learning!
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=2, batch_size=128)
score = model.evaluate(x_val, y_val, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

### Try Word2Vec with Glove

In [3]:
glove_input_file = './glove.6B/glove.6B.100d.txt'
word2vec_output_file = './glove.6B/glove.6B.100d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)

(400000, 100)

In [4]:
# load the Stanford GloVe model
filename = './glove.6B/glove.6B.100d.txt.word2vec'
model = KeyedVectors.load_word2vec_format(filename, binary=False)
# calculate: (king - man) + woman = ?
result = model.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)

[('queen', 0.7698541283607483)]


In [8]:
print(model)
#print(list(model.wv.vocab))

In [9]:
sentences = [['this', 'is', 'the', 'first', 'sentence', 'for', 'word2vec'],
             ['this', 'is', 'the', 'second', 'sentence'],
             ['yet', 'another', 'sentence'],
             ['one', 'more', 'sentence'],
             ['and', 'the', 'final', 'sentence']]

In [13]:
print(len(model['samsung']))

100
